In [2]:
import requests

import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import re

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [3]:
url = "http://www.gutenberg.org/cache/epub/14726/pg14726.txt"
raw_text = requests.get(url).content.decode('utf-8')
test = re.findall("\n +(.*[^ ]) {2,}\d+", raw_text)
eddas = {}
chapters = [chapter.upper() for chapter in re.findall("\n +(.*[^ ]) {2,}\d+", raw_text)][:-2]
raw_text = re.sub("((\n[A-ZÀÂÄÖÔÉÈËÊÏÎŸÇÙÛÜÆŒ][A-ZÀÂÄÖÔÉÈËÊÏÎŸÇÙÛÜÆŒ \.\'\-]+\.)|(ODIN BEGUILES THE DAUGHTER OF BAUGI))", "NEW CHAPTER\n\\1\n", raw_text)
raw_text = re.sub("_([a-zA-ZäöüàâäôéèëêïîçùûüÿæœÀÂÄÖÔÉÈËÊÏÎŸÇÙÛÜÆŒ]+)\.?_\.?\r\n(\r\n\d+\. )([\"A-Za-z])", "\\2\\1: \\3", raw_text, flags=re.UNICODE)
chapters_split = raw_text.split("NEW CHAPTER")[16:-32]
for i in range(0, len(chapters)):
    eddas[chapters[i]] = re.split("\r\n\d+\.", chapters_split[i])[1:]
    for each_verse in range(0, len(eddas[chapters[i]])):
        eddas[chapters[i]][each_verse] = re.sub("[\r\n]+", " ", eddas[chapters[i]][each_verse]).strip()
        eddas[chapters[i]][each_verse] = re.sub("(\[.*\])|( FOOTNOTES:)", "", eddas[chapters[i]][each_verse])

In [4]:
sent_tokens = {}
word_tokens = {}
for chapter in chapters:
    sent_tokens[chapter] = [nltk.sent_tokenize(verse) for verse in eddas[chapter]]
    sent_tokens[chapter] = [sentence for sentences in sent_tokens[chapter] for sentence in sentences]
    word_tokens[chapter] = [nltk.word_tokenize(verse) for verse in eddas[chapter]]
    word_tokens[chapter] = [word for words in word_tokens[chapter] for word in words]

sent_tokens_list = [sentence for current_list in list(sent_tokens.values()) for sentence in current_list]
elder_sent_tokens_list = [sentence for current_list in list(sent_tokens.values())[:37] for sentence in current_list]
younger_sent_tokens_list = [sentence for current_list in list(sent_tokens.values())[37:] for sentence in current_list]
word_tokens_list = [word for current_list in list(word_tokens.values()) for word in current_list]

In [39]:
"""def word_features(words):
    return dict([(word, True) for word in words])

negative_words = movie_reviews.fileids("neg")
negative_features = [(word_features(movie_reviews.words(fileids=[feature])), "negative") for feature in negative_words]
positive_words = movie_reviews.fileids('pos')
positive_features = [(word_features(movie_reviews.words(fileids=[feature])), "positive") for feature in positive_words]

max_features = {
    "negative": int(len(negative_features)*.80),
    "positive": int(len(positive_features)*.80)
}

classifier_train_features = negative_features[:max_features["negative"]] + positive_features[:max_features["positive"]]
classifier_test_features = negative_features[max_features["negative"]:] + positive_features[max_features["positive"]:]
initial_classifier = NaiveBayesClassifier.train(classifier_train_features)
print(nltk.classify.util.accuracy(initial_classifier, classifier_test_features))
initial_classifier.show_most_informative_features()"""

["THE VALA'S PROPHECY",
 'THE LAY OF VAFTHRUDNIR',
 "THE LAY OF VEGTAM, OR BALDUR'S DREAM",
 "THE HIGH ONE'S LAY",
 "ODIN'S RUNE SONG",
 'THE LAY OF HYMIR',
 'THE LAY OF THRYM, OR THE HAMMER RECOVERED',
 'THE LAY OF THE DWARF ALVIS',
 'THE LAY OF HARBARD',
 'THE JOURNEY, OR LAY OF SKIRNIR',
 'THE LAY OF RIG',
 "EGIR'S COMPOTATION, OR LOKI'S ALTERCATION",
 'THE LAY OF FIOLSVITH',
 'THE LAY OF HYNDLA',
 'THE INCANTATION OF GROA',
 'THE SONG OF THE SUN',
 'THE LAY OF VOLUND',
 "THE LAY OF HELGI HIORVARD'S SON",
 'THE FIRST LAY OF HELGI HUNDINGCIDE',
 'THE SECOND LAY OF HELGI HUNDINGCIDE',
 "SINFIOTLI'S END",
 "THE LAY OF SIGURD, OR GRIPIR'S PROPHECY",
 'THE LAY OF FAFNIR',
 'THE LAY OF SIGRDRIFA',
 'FRAGMENTS OF THE LAY OF SIGURD AND BRYNHILD',
 'THE THIRD LAY OF SIGURD FAFNICIDE',
 'FRAGMENTS OF THE LAY OF BRYNHILD',
 'THE FIRST LAY OF GUDRUN',
 "BRYNHILD'S HEL-RIDE",
 'THE SLAUGHTER OF THE NIFLUNGS',
 'THE SECOND LAY OF GUDRUN',
 'THE THIRD LAY OF GUDRUN',
 "ODDRUN'S LAMENT",
 'THE LAY 

In [6]:
"""next_classifier = NaiveBayesClassifier.train(negative_features + positive_features)
#print(nltk.classify.util.accuracy(next_classifier, word_tokens_list))
sentiment_analyzer = SentimentAnalyzer()
all_words_negative = sentiment_analyzer.all_words([mark_negation(sentence) for sentence in elder_sent_tokens_list])"""

'next_classifier = NaiveBayesClassifier.train(negative_features + positive_features)\n#print(nltk.classify.util.accuracy(next_classifier, word_tokens_list))\nsentiment_analyzer = SentimentAnalyzer()\nall_words_negative = sentiment_analyzer.all_words([mark_negation(sentence) for sentence in elder_sent_tokens_list])'

In [7]:
"""unigram_word_features = sentiment_analyzer.unigram_word_feats(all_words_negative, min_freq=4)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_word_features)
training = sentiment_analyzer.apply_features(elder_sent_tokens_list)
testing = sentiment_analyzer.apply_features(younger_sent_tokens_list)"""

'unigram_word_features = sentiment_analyzer.unigram_word_feats(all_words_negative, min_freq=4)\nsentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_word_features)\ntraining = sentiment_analyzer.apply_features(elder_sent_tokens_list)\ntesting = sentiment_analyzer.apply_features(younger_sent_tokens_list)'

In [49]:
sia = SentimentIntensityAnalyzer()
scores = [sia.polarity_scores(sentence) for sentence in sent_tokens_list]
gods = ["Odin", "Thor", "Loki", "Frigga", "Frey", "Freyia", "Bragi", "Ty", "Sif", "Baldr", "Heimdall", "Delling", "Sigyn"]
giants = ["Oegir", "Gymir", "Utgard-Loki"]

def god_scores(godname):
    if godname != "Loki":
        list_of_sentences = [sentence if len(re.findall(godname, sentence)) > 0 else None for sentence in sent_tokens_list]
    else:
        list_of_sentences = [sentence if len(re.findall("[^-]Loki", sentence)) > 0 else None for sentence in sent_tokens_list]
    list_of_sentences = list(filter(None, list_of_sentences))
    god_scores = [scores[sent_tokens_list.index(sentence)] for sentence in list_of_sentences]
    return({"sentences": list_of_sentences, "scores": god_scores})

gods_df = {}
giants_df = {}

for god in gods:
    gods_df[god] = god_scores(god)

for giant in giants:
    giants_df[giant] = god_scores(giant)
    
raw_myth_df = {
    "name": [],
    "status": [],
    "sentence": [],
    "positive": [],
    "neutral": [],
    "negative": [],
    "pos-neg": []
}

for god in gods:
    raw_myth_df["name"] += [god]*len(gods_df[god]["scores"])
    raw_myth_df["status"] += ["God"]*len(gods_df[god]["scores"])
    raw_myth_df["sentence"] += gods_df[god]["sentences"]
    for score in gods_df[god]["scores"]:
        raw_myth_df["positive"] += [score["pos"]]
        raw_myth_df["neutral"] += [score["neu"]]
        raw_myth_df["negative"] += [score["neg"]]
        raw_myth_df["pos-neg"] += [score["pos"]-score["neg"]]

for giant in giants:
    raw_myth_df["name"] += [giant]*len(giants_df[giant]["scores"])
    raw_myth_df["status"] += ["Jotunn"]*len(giants_df[giant]["scores"])
    raw_myth_df["sentence"] += giants_df[giant]["sentences"]
    for score in giants_df[giant]["scores"]:
        raw_myth_df["positive"] += [score["pos"]]
        raw_myth_df["neutral"] += [score["neu"]]
        raw_myth_df["negative"] += [score["neg"]]
        raw_myth_df["pos-neg"] += [score["pos"]-score["neg"]]

In [50]:
myth_df = pd.DataFrame(raw_myth_df)
myth_df

,name,status,sentence,positive,neutral,negative,pos-neg
0,Odin,God,"Spirit gave Odin, sense gave Hoenir, blood gav...",0.134,0.866,0.000,0.134
1,Odin,God,Odin!,0.000,1.000,0.000,0.000
2,Odin,God,"Odin cast , and mid the people hurled it: that...",0.000,0.872,0.128,-0.128
3,Odin,God,"I saw of Baldr, the blood-stained god, Odin's ...",0.174,0.826,0.000,0.174
4,Odin,God,"Loud blows Heimdall, his horn is raised; Odin ...",0.000,1.000,0.000,0.000
5,Odin,God,"Then arises Hlîn's second grief, when Odin goe...",0.109,0.674,0.217,-0.108
6,Odin,God,Then comes the mighty son of Hlôdyn: (Odin's s...,0.000,0.865,0.135,-0.135
7,Odin,God,"Odin: Counsel thou me now, Frigg!",0.000,1.000,0.000,0.000
8,Odin,God,"Odin: Much have I journeyed, much experienced,...",0.000,1.000,0.000,0.000
9,Odin,God,Then went Odin the lore to prove of that all-w...,0.000,1.000,0.000,0.000


In [51]:
alignments = pd.DataFrame({"name": gods+giants})
alignments["avg_positive"] = [np.mean(myth_df[myth_df["name"]==name]["positive"]) for name in alignments["name"]]
alignments["avg_neutral"] = [np.mean(myth_df[myth_df["name"]==name]["neutral"]) for name in alignments["name"]]
alignments["avg_negative"] = [np.mean(myth_df[myth_df["name"]==name]["negative"]) for name in alignments["name"]]
for column in list(myth_df.columns)[-4:-1]:
    column_z = "z_" + column
    alignments[column_z] = (myth_df[column] - myth_df[column].mean())/myth_df[column].std(ddof=0)
alignments["avg_pos-neg"] = [np.mean(myth_df[myth_df["name"]==name]["pos-neg"]) for name in alignments["name"]]

In [52]:
alignments

,name,avg_positive,avg_neutral,avg_negative,z_positive,z_neutral,z_negative,avg_pos-neg
0,Odin,0.058746,0.900246,0.041008,0.613837,-0.061685,-0.593146,0.017738
1,Thor,0.054323,0.887478,0.058236,-0.695573,0.949993,-0.593146,-0.003913
2,Loki,0.051344,0.889062,0.059625,-0.695573,-0.016386,0.795438,-0.008281
3,Frigga,0.099882,0.862882,0.037353,1.004706,-0.363678,-0.593146,0.062529
4,Frey,0.081029,0.850912,0.068074,-0.695573,0.949993,-0.593146,0.012956
5,Freyia,0.059840,0.866320,0.073840,0.369544,-1.511253,1.760938,-0.014000
6,Bragi,0.130640,0.834600,0.034800,-0.695573,-0.069235,0.871376,0.095840
7,Ty,0.174600,0.751000,0.074450,-0.695573,0.949993,-0.593146,0.100150
8,Sif,0.048750,0.925000,0.026250,-0.695573,0.949993,-0.593146,0.022500
9,Baldr,0.100500,0.859786,0.039786,-0.695573,0.949993,-0.593146,0.060714
